In [27]:
import numpy, os
import torch
torch.set_printoptions()
numpy.set_printoptions(suppress=True)
import round

ModuleNotFoundError: No module named 'round'

In [18]:
img_shape = [96,96]

label_path = '/media/hdd/lingjia/hdd_rpsf/data/plain/train/0620_uniformFlux/label.txt'
root_dir = '/media/hdd/lingjia/hdd_rpsf/data/plain/train/0620_uniformFlux'
# list_IDs = [int(n.strip('im').strip('.mat')) for n in os.listdir(os.path.join(root_dir,'noise')) if 'im' in n]

# label_gen
label_raw_intial = numpy.loadtxt(label_path)
if label_raw_intial.ndim < 2:
    label_raw = numpy.expand_dims(label_raw_intial, axis=0)

# given selected ID list
list_IDs = [1,2,3]
i_bol = numpy.zeros(label_raw_intial.shape[0],dtype=numpy.int8)
for idx in list_IDs:
    i_bol += label_raw_intial[:,0] == idx

label_raw = label_raw_intial[i_bol>0,:]
print(f'label_raw_initial: {label_raw.shape} label_raw: {label_raw.shape}')
print(f'label_raw_partial:\n{label_raw[:3]}')

# Initialize frame_ix
label_raw[:,0] = label_raw[:,0] - numpy.min(label_raw[:,0])

labels = torch.tensor(label_raw[:,[0,4,2,1,3]]).float() # frame_idx, flux, x, y, zeta
labels[:,2] = labels[:,2] + img_shape[0]/2 #x
labels[:,3] = labels[:,3] + img_shape[1]/2 #y
print(f'label_raw_partial_initialized:\n{labels[:3]}')


label_raw_initial: (55, 5) label_raw: (55, 5)
label_raw_partial:
[[  1.      12.5949  14.9821  18.7305 139.8652]
 [  1.     -20.0972 -33.9922  -7.463   88.809 ]
 [  1.      25.712  -13.4414   7.6929  99.7835]]
label_raw_partial_initialized:
tensor([[  0.0000, 139.8652,  62.9821,  60.5949,  18.7305],
        [  0.0000,  88.8090,  14.0078,  27.9028,  -7.4630],
        [  0.0000,  99.7835,  34.5586,  73.7120,   7.6929]])


In [21]:
"""BEFORE ADD NEW GT FORMAT"""
idx = 0
n_max = 15
tars = labels
n_frames = 1

param_tar = torch.zeros((n_frames, n_max, 4))
mask_tar = torch.zeros((n_frames, n_max)).bool()

i_bol = tars[:,0] == idx
tar = tars[i_bol,1:]
n_emitter = len(tar)

if n_emitter > n_max:
    raise ValueError(f"Number of actual emitters {n_emitter} exceeds number of max. emitters {n_max} for sample {idx}.")

mask_tar[0, :n_emitter] = 1
param_tar[0, :n_emitter, :] = tar
# bg = torch.zeros(img_shape)
bg = torch.ones(img_shape)*5

mask_tar = mask_tar.squeeze()
param_tar = param_tar.squeeze()

print(f'target_pxyz:\n{tar}')


target pxyz:
tensor([[139.8652,  62.9821,  60.5949,  18.7305],
        [ 88.8090,  14.0078,  27.9028,  -7.4630],
        [ 99.7835,  34.5586,  73.7120,   7.6929],
        [121.3040,  23.9794,  15.8624,  15.0556],
        [148.6605,  20.2790,  59.5918,  15.7843],
        [119.9110,  26.6657,  42.3767, -16.5982],
        [ 81.4084,  37.4981,  51.9909, -18.4378],
        [136.8806,  40.9802,  23.5463, -13.2068],
        [ 96.3204,  50.6395,  27.4709,  15.1257],
        [137.2270,  42.5052,  68.4506, -16.0661],
        [107.7235,  64.1392,  59.0078,  17.6883],
        [141.5032,  62.6383,  61.3725,  17.0140],
        [132.9189,  64.4222,  60.7503,  19.7869]])


In [44]:
"""NEW GT FORMAT"""
prob_map = torch.zeros(img_shape)
print(f'prob_map shape:{prob_map.shape}')

param_tar1 = torch.zeros((n_frames, n_max, 4))
mask_tar1 = torch.zeros((n_frames, n_max)).bool()
param_tar2 = torch.zeros((n_frames, n_max, 4))
mask_tar2 = torch.zeros((n_frames, n_max)).bool()

# for pt_id in range(n_emitter):
#     x = tar[pt_id, 1]
#     y = tar[pt_id, 2]
#     prob_map1[round(float(x)),round(float(y))] += 1

# print(f'max in prob_map1: {torch.max(prob_map1)} {prob_map1.dtype}')

# prob_map2 = (prob_map1>1).to(torch.float32)
# print(f'max in prob_map2: {torch.max(prob_map2)} {prob_map2.dtype}')

mask1_idx = 0
mask2_idx = 0
for pt_id in range(n_emitter):
    x = tar[pt_id, 1]
    y = tar[pt_id, 2]
    prob_map[round(float(x)),round(float(y))] += 1

    if prob_map[round(float(x)),round(float(y))] == 1:
        mask_tar1[0,mask1_idx] = 1
        param_tar1[0,mask1_idx,:] = tar[pt_id]
        mask1_idx += 1
    else:
        mask_tar2[0,mask2_idx] = 1
        param_tar2[0,mask2_idx,:] = tar[pt_id]
        mask2_idx += 1

print(mask_tar1)
print(mask_tar2)
print(param_tar1)
print(param_tar2)
print(param_tar)


prob_map shape:torch.Size([96, 96])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False]])
tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False]])
tensor([[[139.8652,  62.9821,  60.5949,  18.7305],
         [ 88.8090,  14.0078,  27.9028,  -7.4630],
         [ 99.7835,  34.5586,  73.7120,   7.6929],
         [121.3040,  23.9794,  15.8624,  15.0556],
         [148.6605,  20.2790,  59.5918,  15.7843],
         [119.9110,  26.6657,  42.3767, -16.5982],
         [ 81.4084,  37.4981,  51.9909, -18.4378],
         [136.8806,  40.9802,  23.5463, -13.2068],
         [ 96.3204,  50.6395,  27.4709,  15.1257],
         [137.2270,  42.5052,  68.4506, -16.0661],
         [107.7235,  64.1392,  59.0078,  17.6883],
         [132.9189,  64.4222,  60.7503,  19.7869],
         [  0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000],
